In [ ]:
import geopandas as gpd
import pandas as pd
import folium
import random
import shapely
import xyzservices.providers as xyz
from folium import plugins
import os
import branca

In [ ]:
gdf = gpd.read_file('./locations.geojsonl.json').rename(columns={'fid':'photo_id'})

gdf = gdf.to_crs('EPSG:3310')

In [ ]:
#Add the Stadia Maps Stamen Toner provider details via xyzservices
tile_provider = xyz.Stadia.StamenTonerLite

#Update the URL to include the API key placeholder
tile_provider["url"] = tile_provider["url"] + f"?api_key={os.environ['STADIA_API_KEY']}"

#Create the folium TileLayer, specifying the API key
stamen = folium.TileLayer(
    tiles=tile_provider.build_url(api_key=os.environ["STADIA_API_KEY"]),
    attr=tile_provider.attribution,
    name=tile_provider.name,
    max_zoom=tile_provider.max_zoom,
    detect_retina=True
)

In [ ]:
def buffer_map(gdf, col, cmap):
    gdf = gdf.copy()
    gdf.geometry = gdf.buffer(15)
    m = gdf.explore(column = col, cmap = cmap,
    tiles = stamen,
    style_kwds={'fillOpacity': 1})
    plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)

    return m

In [ ]:
buffer_map(gdf, 'points', branca.colormap.step.Dark2_04.scale(vmax=4, vmin=1))

In [ ]:
folium.__version__

## Team Results

In [ ]:
teams = [f'team_{i}' for i in range(1,5)]

In [ ]:
teams

In [ ]:
def read_team(path, team):
    cols = ['team_id', 'photo_id', 'timestamp']
    df = pd.read_excel(path, sheet_name=team)[cols]
    return df.query('photo_id != 99')

In [ ]:
read_team('./photo_scoring.xlsx', 'team_1')

In [ ]:
all_teams = pd.concat([read_team('./photo_scoring.xlsx', team) for team in teams])

### nobody found these

In [ ]:
buffer_map(gdf.query('~ photo_id.isin(@all_teams.photo_id)'), 'points', branca.colormap.step.Dark2_04.scale(vmax=4, vmin=1))

### popular spots

In [ ]:
popular = (all_teams.groupby('photo_id')
 .count()
 .reset_index()
 .rename(columns={'team_id':'team_count'})
 .sort_values('team_count', ascending=False)
 [['photo_id', 'team_count']])

In [ ]:
buffer_map(gdf.merge(popular, on=['photo_id']), 'team_count', branca.colormap.step.YlOrRd_04.scale(vmax=4, vmin=1))

## all_teams

In [ ]:
teams_merged = gdf.merge(all_teams, on='photo_id')

In [ ]:
def jitter(row):
    
    max_jitter = 25
    new_x = row.geometry.x + (max_jitter*random.random())
    new_y = row.geometry.y + (max_jitter*random.random())
    row.geometry = shapely.Point(new_x, new_y)

    return row

In [ ]:
buffer_map(teams_merged.apply(jitter, axis=1), 'team_id',
           branca.colormap.step.Spectral_04.scale(vmax=4, vmin=1))